**URL** - это ссылка на видео из задания  
**frame** - кадр из задания  
**class_object** - посмотри видео и напиши самый большой обьект на английском.

In [ ]:
#@title Что-то меняем...
URL = "https://courses.openedu.ru/assets/courseware/v1/08df7bdc1d63d74835e9797804d5708d/asset-v1:ITMOUniversity+COMPVIS+spring_2022_ITMO+type@asset+block/var_MOT17-10-SDP_2.mp4" #@param {type:"string"}
frame =  73#@param {type:"integer"}
class_object = "airplane" #@param {type:"string"}

video_name = URL.split("/")[-1]
print(f"video_name = {video_name}")

video_name = var_MOT17-10-SDP_2.mp4


Скачай файл видео:

In [ ]:
!wget "https://courses.openedu.ru/assets/courseware/v1/5dab01488703c262d33ed6a5ef074a71/asset-v1:ITMOUniversity+COMPVIS+spring_2022_ITMO+type@asset+block/var_MOT17-04-SDP_4.mp4"

--2022-05-12 12:35:13--  https://courses.openedu.ru/assets/courseware/v1/5dab01488703c262d33ed6a5ef074a71/asset-v1:ITMOUniversity+COMPVIS+spring_2022_ITMO+type@asset+block/var_MOT17-04-SDP_4.mp4
Resolving courses.openedu.ru (courses.openedu.ru)... 178.248.235.109
Connecting to courses.openedu.ru (courses.openedu.ru)|178.248.235.109|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://cdn.openedu.ru/ESZCBLZ/courseware/v1/5dab01488703c262d33ed6a5ef074a71/asset-v1:ITMOUniversity+COMPVIS+spring_2022_ITMO+type@asset+block/var_MOT17-04-SDP_4.mp4 [following]
--2022-05-12 12:35:14--  https://cdn.openedu.ru/ESZCBLZ/courseware/v1/5dab01488703c262d33ed6a5ef074a71/asset-v1:ITMOUniversity+COMPVIS+spring_2022_ITMO+type@asset+block/var_MOT17-04-SDP_4.mp4
Resolving cdn.openedu.ru (cdn.openedu.ru)... 212.193.146.51
Connecting to cdn.openedu.ru (cdn.openedu.ru)|212.193.146.51|:443... failed: Connection refused.


In [ ]:
#@title ПРОСТО ЗАПУСТИ МЕНЯ!!!
!pip install filterpy 1>/dev/null 2>/dev/null

import numpy as np
import cv2
import random

try:
  from numba import jit
except:
  def jit(func):
    return func

np.random.seed(0)

from filterpy.kalman import KalmanFilter

def linear_assignment(cost_matrix):
  try:
    import lap
    _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    return np.array([[y[i],i] for i in x if i >= 0]) #
  except ImportError:
    from scipy.optimize import linear_sum_assignment
    x, y = linear_sum_assignment(cost_matrix)
    return np.array(list(zip(x, y)))


@jit
def iou(bb_test, bb_gt):
  """
  Computes IUO between two bboxes in the form [x1,y1,x2,y2]
  """
  xx1 = np.maximum(bb_test[0], bb_gt[0])
  yy1 = np.maximum(bb_test[1], bb_gt[1])
  xx2 = np.minimum(bb_test[2], bb_gt[2])
  yy2 = np.minimum(bb_test[3], bb_gt[3])
  w = np.maximum(0., xx2 - xx1)
  h = np.maximum(0., yy2 - yy1)
  wh = w * h
  o = wh / ((bb_test[2] - bb_test[0]) * (bb_test[3] - bb_test[1])
    + (bb_gt[2] - bb_gt[0]) * (bb_gt[3] - bb_gt[1]) - wh)
  return(o)


def convert_bbox_to_z(bbox):
  """
  Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
    [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
    the aspect ratio
  """
  w = bbox[2] - bbox[0]
  h = bbox[3] - bbox[1]
  x = bbox[0] + w/2.
  y = bbox[1] + h/2.
  s = w * h    #scale is just area
  r = w / float(h)
  return np.array([x, y, s, r]).reshape((4, 1))


def convert_x_to_bbox(x,score=None):
  """
  Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
    [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
  """
  w = np.sqrt(x[2] * x[3])
  h = x[2] / w
  if(score==None):
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.]).reshape((1,4))
  else:
    return np.array([x[0]-w/2.,x[1]-h/2.,x[0]+w/2.,x[1]+h/2.,score]).reshape((1,5))


class KalmanBoxTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as bbox.
  """
  count = 0
  def __init__(self,bbox):
    """
    Initialises a tracker using initial bounding box.
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=7, dim_z=4) 
    self.kf.F = np.array([[1,0,0,0,1,0,0],[0,1,0,0,0,1,0],[0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0],[0,0,0,0,0,1,0],[0,0,0,0,0,0,1]])
    self.kf.H = np.array([[1,0,0,0,0,0,0],[0,1,0,0,0,0,0],[0,0,1,0,0,0,0],[0,0,0,1,0,0,0]])

    self.kf.R[2:,2:] *= 10.
    self.kf.P[4:,4:] *= 1000. #give high uncertainty to the unobservable initial velocities
    self.kf.P *= 10.
    self.kf.Q[-1,-1] *= 0.01
    self.kf.Q[4:,4:] *= 0.01

    self.kf.x[:4] = convert_bbox_to_z(bbox)
    self.time_since_update = 0
    self.id = KalmanBoxTracker.count
    KalmanBoxTracker.count += 1
    self.history = []
    self.hits = 0
    self.hit_streak = 0
    self.age = 0

  def update(self,bbox):
    """
    Updates the state vector with observed bbox.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1
    self.kf.update(convert_bbox_to_z(bbox))

  def predict(self):
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    if((self.kf.x[6]+self.kf.x[2])<=0):
      self.kf.x[6] *= 0.0
    self.kf.predict()
    self.age += 1
    if(self.time_since_update>0):
      self.hit_streak = 0
    self.time_since_update += 1
    self.history.append(convert_x_to_bbox(self.kf.x))
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return convert_x_to_bbox(self.kf.x)


def associate_detections_to_trackers(detections,trackers,iou_threshold = 0.3):
  """
  Assigns detections to tracked object (both represented as bounding boxes)
  Returns 3 lists of matches, unmatched_detections and unmatched_trackers
  """
  if(len(trackers)==0):
    return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,5),dtype=int)
  iou_matrix = np.zeros((len(detections),len(trackers)),dtype=np.float32)

  for d,det in enumerate(detections):
    for t,trk in enumerate(trackers):
      iou_matrix[d,t] = iou(det,trk)

  if min(iou_matrix.shape) > 0:
    a = (iou_matrix > iou_threshold).astype(np.int32)
    if a.sum(1).max() == 1 and a.sum(0).max() == 1:
        matched_indices = np.stack(np.where(a), axis=1)
    else:
      matched_indices = linear_assignment(-iou_matrix)
  else:
    matched_indices = np.empty(shape=(0,2))

  unmatched_detections = []
  for d, det in enumerate(detections):
    if(d not in matched_indices[:,0]):
      unmatched_detections.append(d)
  unmatched_trackers = []
  for t, trk in enumerate(trackers):
    if(t not in matched_indices[:,1]):
      unmatched_trackers.append(t)

  #filter out matched with low IOU
  matches = []
  for m in matched_indices:
    if(iou_matrix[m[0], m[1]]<iou_threshold):
      unmatched_detections.append(m[0])
      unmatched_trackers.append(m[1])
    else:
      matches.append(m.reshape(1,2))
  if(len(matches)==0):
    matches = np.empty((0,2),dtype=int)
  else:
    matches = np.concatenate(matches,axis=0)

  return matches, np.array(unmatched_detections), np.array(unmatched_trackers)


class Sort(object):
  def __init__(self, max_age=1, min_hits=3):
    """
    Sets key parameters for SORT
    """
    self.max_age = max_age
    self.min_hits = min_hits
    self.trackers = []
    self.frame_count = 0

  def update(self, dets=np.empty((0, 5))):
    """
    Params:
      dets - a numpy array of detections in the format [[x1,y1,x2,y2,score],[x1,y1,x2,y2,score],...]
    Requires: this method must be called once for each frame even with empty detections (use np.empty((0, 5)) for frames without detections).
    Returns the a similar array, where the last column is the object ID.
    NOTE: The number of objects returned may differ from the number of detections provided.
    """
    self.frame_count += 1
    # get predicted locations from existing trackers.
    trks = np.zeros((len(self.trackers), 5))
    to_del = []
    ret = []
    for t, trk in enumerate(trks):
      pos = self.trackers[t].predict()[0]
      trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
      if np.any(np.isnan(pos)):
        to_del.append(t)
    trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
    for t in reversed(to_del):
      self.trackers.pop(t)
    matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets,trks)

    # update matched trackers with assigned detections
    for m in matched:
      self.trackers[m[1]].update(dets[m[0], :])

    # create and initialise new trackers for unmatched detections
    for i in unmatched_dets:
        trk = KalmanBoxTracker(dets[i,:])
        self.trackers.append(trk)
    i = len(self.trackers)
    for trk in reversed(self.trackers):
        d = trk.get_state()[0]
        if (trk.time_since_update < 1) and (trk.hit_streak >= self.min_hits or self.frame_count <= self.min_hits):
          ret.append(np.concatenate((d,[trk.id+1])).reshape(1,-1)) # +1 as MOT benchmark requires positive
        i -= 1
        # remove dead tracklet
        if(trk.time_since_update > self.max_age):
          self.trackers.pop(i)
    if(len(ret)>0):
      return np.concatenate(ret)
    return np.empty((0,5))

import os
from os.path import exists, join, basename, splitext
if not exists('yolov3.weights'):
  !wget -q  https://pjreddie.com/media/files/yolov3.weights
if not exists('yolov3.cfg'):
  !wget -q https://github.com/pjreddie/darknet/raw/master/cfg/yolov3.cfg  
if not exists('coco.names'):
  !wget -q https://github.com/pjreddie/darknet/raw/master/data/coco.names

def load_class_names(namesfile):
    class_names = []
    with open(namesfile, 'r') as fp:
        lines = fp.readlines()
    for line in lines:
        line = line.rstrip()
        class_names.append(line)
    return class_names

In [ ]:
#@title Запусти теперь меня и получи ответы :)
# Загружаем сеть
net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# Инициализируем трекер
mot_tracker = Sort()
# Используем YOLO
namesfile = 'coco.names'
class_names = load_class_names(namesfile)
# Указываем необходимый видео-файл
cap = cv2.VideoCapture(video_name)
# Указываем кодеки для результирующего видео
fourcc = cv2.VideoWriter_fourcc(*'XVID')
videoWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
videoHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# Определяем результирующее видео
out = cv2.VideoWriter('output_SORT.avi',fourcc, 20.0,(videoWidth,videoHeight))
# Генерируем будующим прямоугольникам разные цвета
color_list = []
for j in range(1000):
  color_list.append(((int)(random.randrange(255)),(int)(random.randrange(255)),(int)(random.randrange(255))))

kk = 0 
ret = True
max_detections = 0
max_classes = 0
while ret:
    ret, img = cap.read()
    if ret:
        # Запустим сеть по кадру
        blob = cv2.dnn.blobFromImage(img, 1 / 255.0, (416, 416), swapRB=True, crop=False)
        h,w,_ = img.shape
        net.setInput(blob)
        layerOutputs = net.forward(ln)
        # Разберём все выходы
        boxes=[]
        confidences=[]
        classIDs=[]
        for output in layerOutputs:
            for detection in output:
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if confidence > 0.5:
                    box = detection[0:4] * np.array([w, h, w, h])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

        idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)
        result_img = np.copy(img)
        dets = []
        count_detection = 0
        total_count_detection = 0
        classes = set()
        for j in range(len(idxs)):
            name = class_names[classIDs[idxs[j][0]]]
            if name == class_object:
                count_detection+=1
            total_count_detection += 1
            classes.add(name)
        if total_count_detection > 0:
            detects = np.zeros((count_detection,5))
            count=0
            # Формат, необходимый для трекера
            for j in range(len(idxs)):
                b = boxes[idxs[j][0]]
                name = class_names[classIDs[idxs[j][0]]]
                x1 = int(b[0])
                y1 = int(b[1])
                x2 = int((b[0] + b[2]))
                y2 = int((b[1] + b[3]))
                if name == class_object: # указываем необходимую метку для объектов
                  box = np.array([x1,y1,x2,y2,confidences[idxs[j][0]]])
                  detects[count,:] = box[:]
                  count+=1
                if (kk == frame):
                  print(f"{name} : {confidences[idxs[j][0]]}")
                  print(f"x = {x1}, y = {y1}")
                  print(f"Ширина = {x2 - x1}, Высота = {y2 - y1}\n")
            # Парсим данные трекера
            if len(detects) != 0:
                trackers = mot_tracker.update(detects)
                index = 0
                for d in trackers:
                  # if (kk == frame):
                    # print()
                    # print(name, ": ", end="")
                    # print(confidences[idxs[index][0]])
                    # print((int(d[0]), int(d[1])), (int(d[2] - d[0]), (int(d[3] - d[1]))))
                  result_img = cv2.rectangle(result_img, ((int)(d[0]), (int)(d[1])), ((int)(d[2]), (int)(d[3])), color_list[0], 2)
                  result_img = cv2.putText(result_img, name + "-" + str(round(confidences[idxs[index][0]], 3)), ((int)(d[0]), (int)(d[1]) - 10), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, color_list[0], 2)
                  index += 1
        out.write(result_img) # пишем кадр в результирующее видео
        if (kk == frame):
            cv2.imwrite(f'frame_{frame}.png', result_img)
        kk += 1
        if (total_count_detection > max_detections):
            max_detections = total_count_detection
        if (len(classes) > max_classes):
            max_classes = len(classes)
    else: 
        break
print(f"Frames in video: {kk}")
print(f"Max detections: {max_detections}")
print(f"Max classes: {max_classes}")
cap.release()
out.release()
cv2.destroyAllWindows()

person : 0.9975242018699646
x = 1589, y = 293
Ширина = 189, Высота = 621

person : 0.9887861609458923
x = 1052, y = 316
Ширина = 66, Высота = 187

person : 0.9794845581054688
x = 1804, y = 302
Ширина = 116, Высота = 602

person : 0.9747881889343262
x = 803, y = 348
Ширина = 47, Высота = 110

person : 0.9549583792686462
x = 902, y = 342
Ширина = 72, Высота = 164

person : 0.9539835453033447
x = 723, y = 340
Ширина = 51, Высота = 122

person : 0.9240983128547668
x = 541, y = 348
Ширина = 61, Высота = 115

person : 0.8904576301574707
x = 852, y = 352
Ширина = 36, Высота = 99

bench : 0.8610988855361938
x = 447, y = 645
Ширина = 660, Высота = 405

person : 0.7830045223236084
x = 238, y = 368
Ширина = 49, Высота = 116

bench : 0.5991846919059753
x = 441, y = 429
Ширина = 129, Высота = 80

person : 0.5508096218109131
x = 413, y = 355
Ширина = 32, Высота = 103

Frames in video: 100
Max detections: 15
Max classes: 3
